# FaaS Continuum

## Prelude

### Imports

In [1]:
from os import makedirs

import plotly.io as pio

from util import *

### Configuration

In [2]:
pl.Config.set_fmt_str_lengths(60)
pio.templates.default = "plotly_white"

%load_ext autoreload
%autoreload 2

### Loading the data

In [3]:
raw_fission = pl.read_ndjson("../ddnn/benchmark/fission-newdeploy-min1.json").lazy()
raw_openfaas = pl.read_ndjson("../ddnn/benchmark/openfaas.json").lazy()
fission = normalize_data(raw_fission).collect()
openfaas = normalize_data(raw_openfaas).collect()

### Create output dirs

In [4]:
makedirs("out/ddnn", exist_ok=True)

## Statistical analysis

### Basic stats

In [5]:
stats_openfaas: pl.DataFrame = get_stats(openfaas)
stats_fission: pl.DataFrame = get_stats(fission)
stats = pl.concat((stats_fission, stats_openfaas))
stats.write_csv("out/ddnn/stats.csv")
stats

topic,max,min,avg,median,std,samples
str,f64,f64,f64,f64,f64,u32
"""fission.inference.cloud-output""",920.493501,10.310655,430.104251,418.006197,269.861114,200
"""fission.inference.edge-output""",922.803313,7.807814,438.89212,417.61643,269.625005,200
"""openfaas.inference.fog-output""",895.476777,30.871266,439.917257,422.110751,270.980879,56
"""openfaas.inference.edge-output""",926.390378,3.124148,437.344831,417.320176,273.55538,144
"""openfaas.inference.cloud-output""",924.029098,5.353977,429.302019,417.745753,272.579391,200


### Analysing the layers

In [8]:
divided_openfaas = divide_by_topics(openfaas)
divided_fission = divide_by_topics(fission)

fig = plot_latency_box({
    "edge openfaas": divided_openfaas["edge"],
    "edge fission": divided_fission["edge"],
    "fog openfaas": divided_openfaas["fog"],
    "fog fission": divided_fission["fog"],
    "cloud openfaas": divided_openfaas["cloud"],
    "cloud fission": divided_fission["cloud"]
})
fig.write_image("out/ddnn/box.pdf")
fig